<a href="https://colab.research.google.com/github/vaguiar/camelyon_dl_2019/blob/data-exploration/training_at_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip  install tf-nightly-2.0-preview

     |████████████████████████████████| 87.5MB 262kB/s 
     |████████████████████████████████| 430kB 35.7MB/s 
     |████████████████████████████████| 61kB 15.6MB/s 
     |████████████████████████████████| 3.1MB 35.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/89/67/41/63cbf0f6ac0a6156588b9587be4db5565f8c6d8ccef98202fc
Successfully built wrapt
ERROR: thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
  Found existing installation: wrapt 1.10.11
    Uninstalling wrapt-1.10.11:
      Successfully uninstalled wrapt-1.10.11


In [12]:
import matplotlib.pyplot as plt
import numpy as np
import os

# The code in this notebook should work identically in TF v1 and v2
import tensorflow as tf
import zipfile

import tensorflow.keras.applications as applications
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("You have version", tf.__version__)
assert tf.__version__ >= "2.0" # TensorFlow ≥ 2.0 required

You have version 2.0.0-dev20190511


### Global constants

In [0]:
# Images will be resized to(TARGET_SHAPE, TARGET_SHAPE) as they're read off disk.
TARGET_SHAPE = 150 
BATCH_SIZE = 25

CLASS_MODE = 'categorical'

NUM_OF_CLASSES = 2
CLASS_NAMES = ['tumor', 'non_tumor']

#################
EPOCHS = 50

### Setting up GCP

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
GCP_PROJECT_ID = 'triple-voyage-239123'
GCP_BUCKET_NAME = 'vaa2114_dl_2019'

In [18]:
!gcloud config set project {GCP_PROJECT_ID}

Updated property [core/project].


### Read Data from GCP

In [0]:
TRAIN_DIR = "/tmp/train/"
GCP_TRAIN_DIR = "train/level5/"

!mkdir -p '$TRAIN_DIR'

### Download Level 5 data into Training Dir

In [27]:
# Download the file.
!gsutil -m cp -r gs://{GCP_BUCKET_NAME}/{GCP_TRAIN_DIR} {TRAIN_DIR}

Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_0_108800_tumor_078.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_0_217600_tumor_035.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_0_217600_tumor_057.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_0_96000_tumor_094.tif...
- [4 files][143.2 KiB/143.2 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_102400_12800_tumor_094.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_102400_19200_tumor_094.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_102400_19200_tumor_101.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/le

### Util functions

In [0]:
def getModel(conv_base):
  model = Sequential()
  model.add(conv_base)
  model.add(Flatten())
  model.add(Dropout(0.5))
  model.add(Dense(216, activation='relu'))
  model.add(Dense(NUM_OF_CLASSES, activation='softmax'))
  
  return model

In [0]:
# Compile and build

def trainModel(model, total_train, total_val, 
               train_data_gen, val_data_gen, EPOCHS):
  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc'])

  history = model.fit_generator(
        train_data_gen,
        steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
        epochs=EPOCHS,
        validation_data=val_data_gen,
        validation_steps=int(np.ceil(total_val / float(BATCH_SIZE))),
        verbose=2)
  
  return history